## Общая Информация

### Описание проекта:
    Заказчик GoProtect. Компания занимается развитием сервиса "Мой Чемпион". Цель сервиса помочь спортивным школам фигурного катания и тренерам мониторить прогресс спортсменов с целью дальнейшего их развития.

### Цель:
    Создать решение матчинга названий спортивных школ с эталонными названиями этих школ.
    Одна и та же школа может быть записана по-разному. Например:
       - Республика Татарстан, СШОР Авиатор
       - республика татарстан, "Авиатор"
       - МБУ ДО СШОР "Авиатор", Республика Татарстан
    

## Загрузка данных

In [1]:
#!pip install -U sentence-transformers
#!pip freeze > requirements.txt

#!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sentence_transformers import (
    SentenceTransformer,
    InputExample,
    util, losses
)

pd.set_option('display.max_row', None)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
user_imput_name = pd.read_csv('Примерное написание.csv')
user_imput_name.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [4]:
name_school = pd.read_csv('Школы.csv')
name_school.head()

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


**Вывод:**
- Подготовить данные. Очистить от знаков припинания
- На основе эталонного датасета подготовить обучающий набор данных за счет аугментации.

## Подготовка данных

In [5]:
def cleaning_txt(df:pd.DataFrame)->pd.DataFrame:
    """ Функция на вход получает объект DataFrame с названием спортивных школ и удаляет все лишние знаки.
    """
    
    data = df.copy()
    data['name'] = (
        data['name']
        .replace(r'[^А-Яа-яёЁA-Za-z\s\d]', ' ', regex=True)
        .replace(r'(\s\w{1,2}\s ^[им.\sА-Я.А-Я.])|(\bг\s)', ' ', regex=True)
        .replace(r'\s+', ' ', regex=True)
    )
    
    return data

In [6]:
df = cleaning_txt(user_imput_name)
df.head()

,school_id,name
0,1836,ООО Триумф
1,1836,Москва СК Триумф
2,610,СШОР Надежда Губернии
3,610,Саратовская область ГБУСО СШОР Надежда Губернии
4,609,СШ Гвоздика


In [7]:
def augmentation(data, N:int=5):
    """Функция получает на вход DataFrame и добавляет ошибки в столбце name
    имитирующие опечатки с перестановкой соседних букв."""
    
    df = data.copy()
    one_list = []
    for i in range(len(df)):
        twu_list = []
    
        twu_list.append(df['name'][i] + ' ' + df['region'][i])
        twu_list.append(df['region'][i] + ' ' + df['name'][i])
    

        for n in range(N):
            tmp = list(twu_list[-1])
            i = np.random.randint(len(twu_list[0])-1)
            tmp[i], tmp[i+1] = tmp[i+1], tmp[i]
            twu_list.append(''.join(tmp))
        
        one_list.append(twu_list)
    
    df['aug'] = one_list
    df['name'] = df['name'] + ' ' + df['region']
    
    df, df_aug =df, df.explode('aug')[['school_id','name','aug']].reset_index(drop=True)
    
    return df, df_aug

In [8]:
school, school_aug = augmentation(cleaning_txt(name_school), N=10)
school.head()

,school_id,name,region,aug
0,1,Авангард Московская область,Московская область,"[Авангард Московская область, Московская облас..."
1,2,Авангард Ямало-Ненецкий АО,Ямало-Ненецкий АО,"[Авангард Ямало-Ненецкий АО, Ямало-Ненецкий АО..."
2,3,Авиатор Республика Татарстан,Республика Татарстан,"[Авиатор Республика Татарстан, Республика Тата..."
3,4,Аврора Санкт-Петербург,Санкт-Петербург,"[Аврора Санкт-Петербург, Санкт-Петербург Аврор..."
4,5,Ice Dream Айс Дрим Санкт-Петербург,Санкт-Петербург,"[Ice Dream Айс Дрим Санкт-Петербург, Санкт-Пет..."


**Вывод:**
За счет аугментации увеличили датасет в пять раз добавив в этолонные названия школ ошибки.

## Обучение модели

In [9]:
def model(train, test):
    """Функция получает на вход тренировочный и тестовый DataFrame.
    Обучает модель и предсказывает school_id.
    Возвращает два объекта: 1)Словарь с accuracy для лучшего результата и топ5 результатов.
    2)Измененный тестовый DataFrame с предсказанными school_id"""
    
    train = train.copy()
    test = test.copy()
    
    model = SentenceTransformer('sentence-transformers/LaBSE')
    
    corpus = model.encode(train.name.values)
    query = model.encode(test.name.values)
    
    search_result = util.semantic_search(query, corpus, top_k=5)
    
    school_id = train.school_id.values
    predict_ind = [row[0]['corpus_id'] for row in search_result]
    test['predict_id'] = train.school_id.values[predict_ind]
    top_5 = [[row[x]['corpus_id'] for x in range(5)] for row in search_result]
    test['top_5_id'] = [[train.school_id[row[x]]for x in range(5)] for row in top_5]
    
    
    score = {
        'top':round((test.school_id == test.predict_id).mean(), 2),
        'top_5':round((test.apply(lambda x: x['school_id'] in  x['top_5_id'], axis=1)).mean(), 2)
    }
    
    return score, test

In [10]:
score, test = model(school, user_imput_name)

print('accuracy для top и top_5 результатов модели: ', score)
test.head()

accuracy для top и top_5 результатов модели:  {'top': 0.67, 'top_5': 0.92}


,school_id,name,predict_id,top_5_id
0,1836,"ООО ""Триумф""",1836,"[1836, 303, 225, 221, 135]"
1,1836,"Москва, СК ""Триумф""",1836,"[1836, 96, 303, 251, 104]"
2,610,"СШОР ""Надежда Губернии",610,"[610, 609, 183, 37, 24]"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",610,"[610, 37, 72, 194, 216]"
4,609,"""СШ ""Гвоздика""",609,"[609, 173, 181, 185, 610]"


## Вывод:
- Лучший результат модели показывает результат в 67% правильно предсказанных значений.
- В топ 5 результатов модели попадают 92% верно предсказанных school_id.
- Для улучшения результатов модели необходимо поработать с аугментацией обучающего датафрейма. Также до обучить модель на размеченном датафрейме на хорошие и плохие примеры.